In [1]:
pip install torch

In [2]:
pip install torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)


100%|██████████| 26421880/26421880 [00:01<00:00, 19065354.70it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 308375.51it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5525766.94it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4786583.24it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(784, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x.view(-1, 784))


In [ ]:
generator = Generator()
discriminator = Discriminator()

criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002)


In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        # Train Discriminator
        real_labels = torch.ones(images.size(0), 1)
        fake_labels = torch.zeros(images.size(0), 1)

        outputs = discriminator(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        z = torch.randn(images.size(0), 100)
        fake_images = generator(z)
        outputs = discriminator(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        d_loss = d_loss_real + d_loss_fake
        optimizer_d.zero_grad()
        d_loss.backward()
        optimizer_d.step()

        # Train Generator
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)

        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}')

    if (epoch+1) % 10 == 0:
        save_image(fake_images, f'fake_images_{epoch+1}.png')


Epoch [1/50], d_loss: 0.07571563124656677, g_loss: 8.023153305053711
Epoch [2/50], d_loss: 0.4975324869155884, g_loss: 2.8558247089385986
Epoch [3/50], d_loss: 0.2196296900510788, g_loss: 2.7006988525390625
Epoch [4/50], d_loss: 0.2318534255027771, g_loss: 3.129263401031494
Epoch [5/50], d_loss: 0.321120947599411, g_loss: 2.6653733253479004
Epoch [6/50], d_loss: 0.7022336721420288, g_loss: 4.1879706382751465
Epoch [7/50], d_loss: 0.1618805229663849, g_loss: 3.821033477783203
Epoch [8/50], d_loss: 0.4705049991607666, g_loss: 3.326803684234619
Epoch [9/50], d_loss: 0.4955553412437439, g_loss: 4.745726585388184
Epoch [10/50], d_loss: 0.5375604033470154, g_loss: 4.719780921936035
Epoch [11/50], d_loss: 0.352304071187973, g_loss: 4.348631858825684
Epoch [12/50], d_loss: 0.17403265833854675, g_loss: 5.401487350463867
Epoch [13/50], d_loss: 0.6686909794807434, g_loss: 3.6096272468566895
Epoch [14/50], d_loss: 0.26058197021484375, g_loss: 3.4947125911712646
Epoch [15/50], d_loss: 0.25780135393